In [1]:
# for Mathematical and Dataframe Operations
import numpy as np
import pandas as pd
# for Data Visualizations
import seaborn as sns
import matplotlib.pyplot as plt
#for pre-processing & splitting data
from sklearn.preprocessing import StandardScaler, MinMaxScaler # values 0 to 1
from sklearn.model_selection import train_test_split, KFold , cross_val_score


For the empty value in the 'Name' column, I tried to derive the name from the 'Email' column. By retrieving the first and last names from the Email column and storing them in the 'Name' column. Then for the rows where the 'Name' and 'Email' column was empty, I removed them from the dataset.
For the empty values in the 'Age' column, I filled it with the mean value of the entire 'Age' column. But I would have preferred to fill the empty 'Age' value with the median of the column, as for continuous values, the median gets less affected by the outliers than the mean value.
For the 'Phone' column, I standardize the column format into (91-xxxxxxxx). So I used the .split('-') method on the 'Phone' column to get the code and number parts of the phone number. Then made sure that the number part only had eight digits. Then the code and number were separated by '-.'
For the rows with empty 'Email' values and no 'Name' value, I removed them from the dataset as we couldn't get the person's name out of it. Whereas for the rows with no 'Email' value but had 'Name' value, fill it with support@dataisgood.com.
For the 'Department' column, make the data properly with the valuable functions and rename the wrong-spelled words in the department column. First, I standardized the text format of the column by removing extra space and keeping all in lower. Here the 'sles' is an obvious spelling error so we change it to 'sales'. 'operaryion' is an obvious spelling error so we change it to 'operations', 'it department' and 'it devlopement' can be referred to as 'IT'. Now we have seven unique values for the 'Department 'column.
For the 'Address' column, replace the null values with 'Address not Available.'
In the end, I made a few extra changes where I changed the type of the 'Salary' column to float the 'Age' column to int, for 'Gender 'and 'Department' datatype to category. We can see that column Id has only unique values, which will not help us in prediction, so I removed this column.

In [2]:
df=pd.read_excel("Unmessenger Assignment -2.xlsm")
df.head()

,Id,Name,Age,Gender,Department,Email,Phone,Address,Salary
0,1,John Doe,32.0,M,Operations,john.doe@gmail.com,91-1234567890,"123 Main St, Bangalore, Karnataka, India",4553
1,2,Jane Smith,28.0,F,IT Department,jane.smith@hotmail.com,91-9876543210,"456 Park Ave, Mumbai, Maharashtra, India",31169
2,3,Amit Patel,27.0,M,IT DeVlopemENT,amit.patel@gmail.com,91-8765432109,"789 3rd St, Kolkata, West Bengal, India",11116
3,4,Priya Sharma,33.0,F,SALES,priya.sharma@yahoo.com,91-5555555555,"12 4th St, New Delhi, Delhi, India",33457
4,5,NaN,29.0,F,SLES,jane.doe@gmail.com,NaN,NaN,21918


In [4]:
df.shape

(50, 9)

In [15]:
Categorical_columns=df.dtypes[df.dtypes=='object'].index
Categorical_columns

Index(['Name', 'Gender', 'Department', 'Email', 'Phone', 'Address'], dtype='object')

Task 1
Where Name is missing fill the value in place by extracting the name from the Email-id
column.

In [5]:
#before any cleaning number of null values in Name column
df['Name'].isnull().sum()

11

In [6]:
#before any cleaning number of null values in Email column
df['Email'].isnull().sum()

3

In [7]:
# number of rows with no name and no email
index_of_missing_name_email = np.where((df['Name'].isnull()==True) & (df['Email'].isnull()==True))
len(index_of_missing_name_email[0])

2

In [7]:
# missing = np.where(df["Name"].isnull() == True)
# # df.loc[missing,"Name"]
# df.iloc[missing]["Email"].str.split('.')

In [8]:
# retrieving first name and last name form the Email column and storing it in Name  column
first_name = df['Email'].str.split(".", expand = True)[0].str.capitalize()
last_name = df['Email'].str.split(".", expand = True)[1].str.split("@", expand = True)[0].str.capitalize()
# print(last_name)
fill_name = np.where(df["Name"].isnull(), # Logical check
                       first_name+" "+last_name                   # Value if check is true
                        ,df["Name"]
                      )     # Value if check is false
fill_name

array(['John Doe', 'Jane Smith', 'Amit Patel', 'Priya Sharma', 'Jane Doe',
       'Rakesh Kumar', 'Divya Singh', 'Aman Verma', 'Neha Gupta',
       'Naveen Sharma', 'Vinita Kapoor', 'Rajeev Kumar', nan,
       'Swati Singh', 'Kavita Sharma', 'Manish Patel', 'Priyanka Jain',
       'Rohit Sharma', 'Deepika Singh', 'Vikas Joshi', 'Nisha Verma',
       'Manoj Kumar', nan, 'Sanjay Gupta', 'Anjali Sharma', 'Akash Patel',
       'Preeti Singh', 'Ravi Kumar', 'Swati Mishra', 'Ankit Singh',
       'Neha Gupta', 'Rajeev Sharma', 'Sarika Patel', 'Vinay Singh',
       'Priya Gupta', 'Sushant Patel', 'Payal Sharma', 'Mohit Verma',
       'Kavita Singh', 'Ankit Patel', 'Pooja Mishra', 'Mohan Kumar',
       'Anjali Gupta', 'Mukesh Sharma', 'Arjun Singh', 'Divya Patel',
       'Gaurav Kumar', 'Aditi Shah', 'Sandeep Singh', 'Shalini Sharma'],
      dtype=object)

In [9]:
df["Name"] = fill_name

In [10]:
# to check number of null values in Name column
df['Name'].isna().sum()

2

In [11]:
# we can see from above that there are still 2 names that are empty that is 
# because the Email column is also empty
index_missing_name = np.where(df["Name"].isnull() == True)
df.loc[index_missing_name,"Name"]
df.iloc[index_missing_name]["Email"]

12    NaN
22    NaN
Name: Email, dtype: object

Task 2
Fill the Missing Age value with the mean of the Age column

In [12]:
# to check number of null values in Age column before cleaning
df['Age'].isna().sum()

4

In [13]:
df['Age']=df['Age'].fillna(df['Age'].mean())

In [16]:
# to check number of null values in Age column after cleaning
df['Age'].isna().sum()

0

Task 3
Clean and Format the Phone number column and it should show like (91-xxxxxxxx)

In [17]:
# to get the code part and number part from the Phone column , 
#number part should have only 8 digits
#the code and number should be seperated by '-'
phone_num = df['Phone'].str.split('-',expand = True)[1].str[:8]
phone_code = df['Phone'].str.split('-',expand = True)[0]
phone_code
fill_phone_num = np.where(df['Phone'].str.split('-',expand = True)[1].str.len()>8, # Logical check
                       phone_code+"-"+phone_num                  # Value if check is true
                        ,df['Phone'] # Value if check is false
                      )     
fill_phone_num

array(['91-12345678', '91-98765432', '91-87654321', '91-55555555', nan,
       nan, '91-99999999', '91-88888888', '91-77777777', nan,
       '91-66666666', '91-55555555', '91-77777777', nan, '91-88888888',
       '91-12345678', '91-87654321', '91-98765432', '91-55555555',
       '91-99999999', nan, '91-88888888', '91-77777777', '91-66666666',
       '91-55555555', nan, '91-77777777', '91-88888888', '91-77777777',
       nan, '91-55555555', nan, '91-66666666', nan, '91-55555555',
       '91-77777777', nan, '91-88888888', '91-99999999', nan,
       '91-77777777', nan, '91-55555555', nan, '91-88888888',
       '91-77777777', nan, '91-66666666', '91-55555555', nan],
      dtype=object)

In [18]:
df['Phone']=fill_phone_num

In [19]:
df['Phone'].isnull().sum()

15

In [20]:
df['Phone']

0     91-12345678
1     91-98765432
2     91-87654321
3     91-55555555
4             NaN
5             NaN
6     91-99999999
7     91-88888888
8     91-77777777
9             NaN
10    91-66666666
11    91-55555555
12    91-77777777
13            NaN
14    91-88888888
15    91-12345678
16    91-87654321
17    91-98765432
18    91-55555555
19    91-99999999
20            NaN
21    91-88888888
22    91-77777777
23    91-66666666
24    91-55555555
25            NaN
26    91-77777777
27    91-88888888
28    91-77777777
29            NaN
30    91-55555555
31            NaN
32    91-66666666
33            NaN
34    91-55555555
35    91-77777777
36            NaN
37    91-88888888
38    91-99999999
39            NaN
40    91-77777777
41            NaN
42    91-55555555
43            NaN
44    91-88888888
45    91-77777777
46            NaN
47    91-66666666
48    91-55555555
49            NaN
Name: Phone, dtype: object

In [21]:
df

,Id,Name,Age,Gender,Department,Email,Phone,Address,Salary
0,1,John Doe,32.0,M,Operations,john.doe@gmail.com,91-12345678,"123 Main St, Bangalore, Karnataka, India",4553
1,2,Jane Smith,28.0,F,IT Department,jane.smith@hotmail.com,91-98765432,"456 Park Ave, Mumbai, Maharashtra, India",31169
2,3,Amit Patel,27.0,M,IT DeVlopemENT,amit.patel@gmail.com,91-87654321,"789 3rd St, Kolkata, West Bengal, India",11116
3,4,Priya Sharma,33.0,F,SALES,priya.sharma@yahoo.com,91-55555555,"12 4th St, New Delhi, Delhi, India",33457
4,5,Jane Doe,29.0,F,SLES,jane.doe@gmail.com,NaN,NaN,21918
5,6,Rakesh Kumar,34.0,M,DaTa SciENce,rakesh.kumar@yahoo.com,NaN,"345 5th St, Jaipur, Rajasthan, India",33418
6,7,Divya Singh,25.0,F,IT DeVlopemENT,NaN,91-99999999,"678 6th St, Chennai, Tamil Nadu, India",45828
7,8,Aman Verma,31.0,M,SALES,aman.verma@gmail.com,91-88888888,NaN,45450
8,9,Neha Gupta,27.0,F,SLES,neha.gupta@hotmail.com,91-77777777,"901 7th St, Hyderabad, Telangana, India",49938
9,10,Naveen Sharma,35.0,M,Operations,naveen.sharma@gmail.com,NaN,"234 8th St, Pune, Maharashtra, India",9818


Task 4
Where Name and Email-Id is missing the delete the entire row.

In [21]:
# df = df.dropna(subset=['Name','Email']) 

In [22]:
# to get index of rows where both Name and Email column are empty
index_of_missing_name_email = np.where((df['Name'].isnull()==True) & (df['Email'].isnull()==True))
index_of_missing_name_email[0]

array([12, 22], dtype=int64)

In [23]:
print("Number of rows with null name and email: ",len(index_of_missing_name_email[0]))

Number of rows with null name and email:  2


In [24]:
# df=df.drop(index_of_missing_name_email,axis=0)
df.drop(index_of_missing_name_email[0],inplace =True)

In [25]:
# to check number of null values in Name column after cleaning
df['Name'].isnull().sum()

0

In [26]:
# to check number of null values in Email column after cleaning
df['Email'].isnull().sum()

1

In [27]:
index_of_missing_name_email = np.where((df['Name'].isnull()==True) & (df['Email'].isnull()==True))
len(index_of_missing_name_email[0])

0

Task 5
Highlight the Top 10 Employees with highest salary in the dataset.

In [28]:
df.sort_values(by='Salary',ascending=False)[:10]

,Id,Name,Age,Gender,Department,Email,Phone,Address,Salary
8,9,Neha Gupta,27.0,F,SLES,neha.gupta@hotmail.com,91-77777777,"901 7th St, Hyderabad, Telangana, India",49938
21,22,Manoj Kumar,33.0,M,Data Science,manoj.kumar@yahoo.com,91-88888888,"901 17th St, Pune, Maharashtra, India",49681
14,15,Kavita Sharma,30.0,F,Finance/Accounts,kavita.sharma@yahoo.com,91-88888888,NaN,49400
36,37,Payal Sharma,31.0,F,SLES,payal.sharma@gmail.com,NaN,"234 30th St, Bangalore, Karnataka, India",48367
6,7,Divya Singh,25.0,F,IT DeVlopemENT,NaN,91-99999999,"678 6th St, Chennai, Tamil Nadu, India",45828
25,26,Akash Patel,31.0,M,Finance/Accounts,akash.patel@hotmail.com,NaN,"789 20th St, Mumbai, Maharashtra, India",45678
7,8,Aman Verma,31.0,M,SALES,aman.verma@gmail.com,91-88888888,NaN,45450
30,31,Neha Gupta,27.0,F,SLES,neha.gupta@gmail.com,91-55555555,NaN,41841
10,11,Vinita Kapoor,29.5,F,IT Department,vinita.kapoor@yahoo.com,91-66666666,"567 9th St, Ahmedabad, Gujarat, India",41836
29,30,Ankit Singh,29.0,M,SALES,ankit.singh@hotmail.com,NaN,"567 24th St, Lucknow, Uttar Pradesh, India",40940


Task 6
Where Name is there but no Email-Id then fill the blank with (support@dataisgood.com)

In [29]:
df['Email'].isnull().sum()

1

In [30]:
df['Email'].fillna('support@dataisgood.com',inplace =True)#impute any value

In [31]:
df

,Id,Name,Age,Gender,Department,Email,Phone,Address,Salary
0,1,John Doe,32.0,M,Operations,john.doe@gmail.com,91-12345678,"123 Main St, Bangalore, Karnataka, India",4553
1,2,Jane Smith,28.0,F,IT Department,jane.smith@hotmail.com,91-98765432,"456 Park Ave, Mumbai, Maharashtra, India",31169
2,3,Amit Patel,27.0,M,IT DeVlopemENT,amit.patel@gmail.com,91-87654321,"789 3rd St, Kolkata, West Bengal, India",11116
3,4,Priya Sharma,33.0,F,SALES,priya.sharma@yahoo.com,91-55555555,"12 4th St, New Delhi, Delhi, India",33457
4,5,Jane Doe,29.0,F,SLES,jane.doe@gmail.com,NaN,NaN,21918
5,6,Rakesh Kumar,34.0,M,DaTa SciENce,rakesh.kumar@yahoo.com,NaN,"345 5th St, Jaipur, Rajasthan, India",33418
6,7,Divya Singh,25.0,F,IT DeVlopemENT,support@dataisgood.com,91-99999999,"678 6th St, Chennai, Tamil Nadu, India",45828
7,8,Aman Verma,31.0,M,SALES,aman.verma@gmail.com,91-88888888,NaN,45450
8,9,Neha Gupta,27.0,F,SLES,neha.gupta@hotmail.com,91-77777777,"901 7th St, Hyderabad, Telangana, India",49938
9,10,Naveen Sharma,35.0,M,Operations,naveen.sharma@gmail.com,NaN,"234 8th St, Pune, Maharashtra, India",9818


In [32]:
# to check number of null values in Email column
df['Email'].isna().sum()

0

Task 7
In the Department column make the data proper with the useful functions and rename the wrong spelled words in the department column.

In [33]:
df['Department'].unique()
#'sles' is an obvious spelling error so we change it to 'sales'
#'operaryion' is an obvious spelling error so we change it to 'operations',
#'it department' and 'it devlopement' can be refered to as 'IT'

array(['Operations', 'IT Department', 'IT DeVlopemENT', 'SALES', 'SLES',
       'DaTa SciENce', 'HR', 'Finance/Accounts', 'Mangement',
       'Data Science', 'hR', 'Hr', 'OperarYION'], dtype=object)

In [34]:
#standardization of text format by removing extra space and keeping all in lower
df['Department']=df['Department'].str.lower().str.strip().astype('category') 

In [35]:
mappings={'sles':'sales','operaryion':'operations','it department':'IT','it devlopement':'IT'}
df['Department'] = df['Department'].replace(mappings) 

In [36]:
df['Department'].unique()

['operations', 'IT', 'sales', 'data science', 'hr', 'finance/accounts', 'mangement']
Categories (7, object): ['data science', 'finance/accounts', 'hr', 'IT', 'mangement', 'operations', 'sales']

Task 8
Where Address is not there fill it with (Address not Available)

In [37]:
df['Address'].isnull().sum()

8

In [38]:
df['Address'].fillna('Adress not Available',inplace =True)#impute any value

In [39]:
df['Address'].isnull().sum()

0

In [40]:
df

,Id,Name,Age,Gender,Department,Email,Phone,Address,Salary
0,1,John Doe,32.0,M,operations,john.doe@gmail.com,91-12345678,"123 Main St, Bangalore, Karnataka, India",4553
1,2,Jane Smith,28.0,F,IT,jane.smith@hotmail.com,91-98765432,"456 Park Ave, Mumbai, Maharashtra, India",31169
2,3,Amit Patel,27.0,M,IT,amit.patel@gmail.com,91-87654321,"789 3rd St, Kolkata, West Bengal, India",11116
3,4,Priya Sharma,33.0,F,sales,priya.sharma@yahoo.com,91-55555555,"12 4th St, New Delhi, Delhi, India",33457
4,5,Jane Doe,29.0,F,sales,jane.doe@gmail.com,NaN,Adress not Available,21918
5,6,Rakesh Kumar,34.0,M,data science,rakesh.kumar@yahoo.com,NaN,"345 5th St, Jaipur, Rajasthan, India",33418
6,7,Divya Singh,25.0,F,IT,support@dataisgood.com,91-99999999,"678 6th St, Chennai, Tamil Nadu, India",45828
7,8,Aman Verma,31.0,M,sales,aman.verma@gmail.com,91-88888888,Adress not Available,45450
8,9,Neha Gupta,27.0,F,sales,neha.gupta@hotmail.com,91-77777777,"901 7th St, Hyderabad, Telangana, India",49938
9,10,Naveen Sharma,35.0,M,operations,naveen.sharma@gmail.com,NaN,"234 8th St, Pune, Maharashtra, India",9818


# my extra step for data cleaning

In [41]:
df['Salary'].astype(float)

0      4553.0
1     31169.0
2     11116.0
3     33457.0
4     21918.0
5     33418.0
6     45828.0
7     45450.0
8     49938.0
9      9818.0
10    41836.0
11    11901.0
13    21978.0
14    49400.0
15    28476.0
16    40088.0
17     2949.0
18     3337.0
19     2857.0
20    35969.0
21    49681.0
23    15853.0
24     6926.0
25    45678.0
26     8022.0
27     3484.0
28    11342.0
29    40940.0
30    41841.0
31    39835.0
32     9577.0
33    29480.0
34    14254.0
35    37358.0
36    48367.0
37    37473.0
38    30628.0
39    22018.0
40    35747.0
41    32258.0
42    31047.0
43    31064.0
44     3108.0
45    20205.0
46     8746.0
47    15531.0
48    23055.0
49    18647.0
Name: Salary, dtype: float64

In [42]:
df['Age'].astype(int)

0     32
1     28
2     27
3     33
4     29
5     34
6     25
7     31
8     27
9     35
10    29
11    29
13    28
14    30
15    32
16    26
17    28
18    30
19    32
20    26
21    33
23    29
24    29
25    31
26    28
27    34
28    26
29    29
30    27
31    32
32    33
33    30
34    29
35    29
36    31
37    28
38    33
39    30
40    27
41    29
42    26
43    28
44    31
45    27
46    29
47    32
48    29
49    30
Name: Age, dtype: int32

In [43]:
df['Gender'].astype('category')

0     M
1     F
2     M
3     F
4     F
5     M
6     F
7     M
8     F
9     M
10    F
11    M
13    F
14    F
15    M
16    F
17    M
18    F
19    M
20    F
21    M
23    M
24    F
25    M
26    F
27    M
28    F
29    M
30    F
31    M
32    F
33    M
34    F
35    M
36    F
37    M
38    F
39    M
40    F
41    M
42    F
43    M
44    M
45    F
46    M
47    F
48    M
49    F
Name: Gender, dtype: category
Categories (2, object): ['F', 'M']

In [44]:
df['Department'].astype('category')

0           operations
1                   IT
2                   IT
3                sales
4                sales
5         data science
6                   IT
7                sales
8                sales
9           operations
10                  IT
11                  IT
13                  hr
14    finance/accounts
15           mangement
16        data science
17        data science
18        data science
19    finance/accounts
20           mangement
21        data science
23          operations
24                  IT
25    finance/accounts
26           mangement
27        data science
28                  IT
29               sales
30               sales
31    finance/accounts
32           mangement
33        data science
34                  IT
35               sales
36               sales
37                  hr
38                  hr
39          operations
40        data science
41               sales
42    finance/accounts
43           mangement
44        data science
45         

In [45]:
len(df['Id'].unique()) 
#we can see that column Id has only unique values which will not help us in prediction so we dont need this column

48

In [46]:
del df['Id']

In [47]:
df

,Name,Age,Gender,Department,Email,Phone,Address,Salary
0,John Doe,32.0,M,operations,john.doe@gmail.com,91-12345678,"123 Main St, Bangalore, Karnataka, India",4553
1,Jane Smith,28.0,F,IT,jane.smith@hotmail.com,91-98765432,"456 Park Ave, Mumbai, Maharashtra, India",31169
2,Amit Patel,27.0,M,IT,amit.patel@gmail.com,91-87654321,"789 3rd St, Kolkata, West Bengal, India",11116
3,Priya Sharma,33.0,F,sales,priya.sharma@yahoo.com,91-55555555,"12 4th St, New Delhi, Delhi, India",33457
4,Jane Doe,29.0,F,sales,jane.doe@gmail.com,NaN,Adress not Available,21918
5,Rakesh Kumar,34.0,M,data science,rakesh.kumar@yahoo.com,NaN,"345 5th St, Jaipur, Rajasthan, India",33418
6,Divya Singh,25.0,F,IT,support@dataisgood.com,91-99999999,"678 6th St, Chennai, Tamil Nadu, India",45828
7,Aman Verma,31.0,M,sales,aman.verma@gmail.com,91-88888888,Adress not Available,45450
8,Neha Gupta,27.0,F,sales,neha.gupta@hotmail.com,91-77777777,"901 7th St, Hyderabad, Telangana, India",49938
9,Naveen Sharma,35.0,M,operations,naveen.sharma@gmail.com,NaN,"234 8th St, Pune, Maharashtra, India",9818


In [54]:
df.to_excel('Garima Cleaned Unmessenger Assignment -2.xlsm')

In [51]:
df.to_csv('Garima Cleaned Unmessenger Assignment -2.csv')